In [1]:
import requests
import time
import pandas as pd
import numpy as np

In [2]:
def get_response(base_url, subreddit):
    return requests.get(base_url)

In [25]:
def get_submissions(subreddits, num_posts):
    posts = pd.DataFrame()
    for i, subreddit in enumerate(subreddits):
        print(f'Pulling posts from subreddit "{subreddit}"...')
        base_url = 'https://api.pushshift.io/reddit/search/submission?limit=100&is_self=true&subreddit=' + '{}'.format(subreddit)
        count = 0
        while len(posts) < num_posts * (i+1): # I want to stop after I get all the posts I need. i+1 will multiply the subreddit count we're on.
            count += 100
            res = get_response(base_url, subreddit)
            res_df = res.json()

            posts = posts.append([res_df['data'][j] for j in range(len(res_df['data']))]) # Appending the new rows to the bottom of the empty "posts" dataframe
            # Update URL with the 'created_utc' value from the last item we pulled. This will give us a starting point for our next 100 posts
            try:
                base_url = 'https://api.pushshift.io/reddit/search/submission?limit=100&is_self=true&subreddit=' + '{}'.format(subreddit) + '&before={}'.format(res_df['data'][99]['created_utc'])
                time.sleep(4) # We want to be nice to the api and not get banned. Could be 3 seconds, but I don't trust time.sleep

                if len(posts) % 200 == 0: # Give me some updates so I know the code is actually working
                    print(f'Pulling {num_posts} posts from subreddit {subreddit}.....{round(count / num_posts * 100, 2)}% complete')
            except:
                print(f'Ran out of posts!')
                break
    
        print('----------------------------------')
        print(f'Finished pulling posts from subreddit "{subreddit}"')
        print('----------------------------------')
    print('Complete!!!!!!!!!!!')
    return posts.reset_index(drop=True)

In [29]:
posts = get_submissions(['punk', 'poppunkers'], 2500)

Pulling posts from subreddit "punk"...
Pulling 2500 posts from subreddit punk.....8.0% complete
Pulling 2500 posts from subreddit punk.....16.0% complete
Pulling 2500 posts from subreddit punk.....24.0% complete
Pulling 2500 posts from subreddit punk.....32.0% complete
Pulling 2500 posts from subreddit punk.....40.0% complete
Pulling 2500 posts from subreddit punk.....48.0% complete
Pulling 2500 posts from subreddit punk.....56.0% complete
Pulling 2500 posts from subreddit punk.....64.0% complete
Pulling 2500 posts from subreddit punk.....72.0% complete
Pulling 2500 posts from subreddit punk.....80.0% complete
Pulling 2500 posts from subreddit punk.....88.0% complete
Pulling 2500 posts from subreddit punk.....96.0% complete
----------------------------------
Finished pulling posts from subreddit "punk"
----------------------------------
Pulling posts from subreddit "poppunkers"...
Pulling 2500 posts from subreddit poppunkers.....4.0% complete
Pulling 2500 posts from subreddit poppunker

In [30]:
posts.shape

(5000, 79)

In [31]:
posts.to_csv('./data/reddit_data.csv', index=False)